In [ ]:
%%bash
pip install apache-airflow s3fs

In [ ]:
%%bash
airflow standalone

In [ ]:
%%writefile example.py
import datetime as dt
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
# Say this is Task1
def greet():
    '''
    A python function to write a text file
    '''
    print('Writing in file')
    with open('/home/parallels/ARIS_Labs/Lab5_AirflowPipelines/greet.txt', 'a+', encoding='utf8') as f:
        now = dt.datetime.now()
        t = now.strftime("%Y-%m-%d %H:%M")
        f.write(str(t) + '\n')
    return 'Greeted'
# Say this is task2
def respond():
    '''
    A python function to return a simple greetingthon function to return a simple greeting
    '''
    return 'Greet Responded Again'
# Declaring DAG default settings
default_args = {
    'owner': 'airflow',
    'start_date': dt.datetime(2018, 9, 24, 10, 00, 00),
    'concurrency': 1,
    'retries': 0
}
# Building the DAG, 'my_simple_dag' is the dag id, which will be
# visible in the airflow ui
with DAG('my_simple_dag',
         catchup=False,
         default_args=default_args,
         schedule_interval='*/10 * * * *',
         # schedule_interval=None,
         ) as dag:
    opr_hello = BashOperator(task_id='say_Hi',
                             bash_command='echo "Hi!!"')
    opr_greet = PythonOperator(task_id='greet',
                               python_callable=greet)
    opr_sleep = BashOperator(task_id='sleep_me',
                             bash_command='sleep 5')
    opr_respond = PythonOperator(task_id='respond',
                                 python_callable=respond)
    # Setting the task flow dependencies
    opr_hello >> opr_greet >> opr_sleep >> opr_respond

In [ ]:
%%writefile iris_learn_pipeline.py
from airflow import DAG
from airflow.decorators import task
from airflow.operators.bash import BashOperator
from airflow.utils.trigger_rule import TriggerRule

import pandas as pd
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
from sklearn.pipeline import Pipeline
from airflow.models import Variable
import s3fs

with DAG('iris_learn_pipeline',
        catchup=False,
        default_args={
            'owner': 'airflow',
            'start_date': datetime(2023, 5, 24),
            'concurrency': 1,
            'retries': 0
        },
        schedule_interval=None) as dag:
    @task
    def read_learn():
        df = pd.read_csv("s3://team1/lab5/iris.csv",
            storage_options={
                "key": os.environ['ACCESS_KEY'],
                "secret": os.environ['SECRET_KEY'],
                "client_kwargs": {"endpoint_url": "http://minio.minio:9000"}
            }
        )
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        dups = X.duplicated()
        duprows = dups.any()
        if duprows:
            X = X[~dups]
            y = y[~dups]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

        os.environ['MLFLOW_TRACKING_USERNAME']='team1'
        os.environ['MLFLOW_TRACKING_PASSWORD']='iKAMiMhK2i'
        mlflow.set_tracking_uri("http://mlflow.team1:5000")
        exps = mlflow.search_experiments(filter_string="name LIKE '%Iris%LAB5%'")
        if len(exps)==0:
            exp_id = mlflow.create_experiment("MLP - Iris dataset - LAB5")
        else:
            exp_id = exps[0].experiment_id
        Variable.set('exp_id', exp_id)

        mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000, verbose=1)

        pipe=Pipeline([('scaler',StandardScaler()),('mlp',mlp)])
        pipe.fit(X_train, y_train)

        schema = infer_signature(X_train, y_train)
        with mlflow.start_run(run_name="Training iris MLP model from Airflow",
                        experiment_id=exp_id) as run:
            Variable.set('run_id', run.info.run_id)
            mlflow.sklearn.log_model(pipe, artifact_path="sklearn-model", signature=schema)
        
            name1="Iris MLP model-Airflow"

            client = MlflowClient()
            rms = client.search_registered_models("name='"+name1+"'")
            if len(rms)==0: client.create_registered_model(name1)

            model_uri = "runs:/{}/sklearn-model".format(run.info.run_id)
            model_src = RunsArtifactRepository.get_underlying_uri(model_uri)
            mv = client.create_model_version(name1, model_src, run.info.run_id, description="Pipeline for iris dataset")

        mlflow.end_run()

    @task
    def generate_yaml():
        exp_id=Variable.get("exp_id")
        run_id=Variable.get("run_id")
        with open('/home/airflow/iris.yaml', 'w+') as f:
            f.write('apiVersion: mlops.seldon.io/v1alpha1\n'+
                        'kind: Model\n'+
                        'metadata:\n'+
                        '   name: iris-lab5\n'+
                        'spec:\n'+
                        f'   storageUri: "s3://team1/mlflow/{exp_id}/{run_id}/artifacts/sklearn-model"\n'+
                        '   secretName: "minio"\n'+
                        '   requirements:\n'+
                        '   - mlflow')
        s3=s3fs.S3FileSystem(key=os.environ['ACCESS_KEY'],secret=os.environ['SECRET_KEY'],endpoint_url="http://minio.minio:9000")
        s3.put('/home/airflow/iris.yaml','s3://team1/lab5/iris.yaml')

    remove = BashOperator(
        task_id='remove_old_model',
        bash_command='kubectl delete -f /home/airflow/iris.yaml'
    )

    deploy = BashOperator(
        task_id='deploy_model',
        bash_command='kubectl apply -f /home/airflow/iris.yaml',
        trigger_rule=TriggerRule.ALL_DONE
    )

    read_learn() >> generate_yaml() >> remove >> deploy